# 미션9

FashionMNIST 데이터셋의 각 패션 아이템(예: 티셔츠, 바지, 스니커즈 등)을 조건부로 생성하는 작업을 수행합니다.

각 클래스에 해당하는 이미지를 생성하는 cGAN (Conditional GAN) 모델을 직접 설계하고 학습시켜 보세요.

**데이터 구성**:
- **훈련 데이터**: 60,000장의 이미지
- **테스트 데이터**: 10,000장의 이미지
- 28×28 크기의 흑백 이미지 **(10개 클래스)**

**클래스 목록**:
- T-shirt/top
- Trouser
- Pullover
- Dress
- Coat
- Sandal
- Shirt
- Sneaker
- Bag
- Ankle boot

### 전체 흐름 요약

1. 데이터 준비 – FashoinMNIST 데이터셋을 다운 받아 정규화(-1~1) 시키고 -> 라벨 One-Hot 또는 임베딩을 준비합니다.
2. DataLoader 정의 - 배치(batch)size, 셔플, 멀티 스레드 로딩합니다.
3. 모델 설계 – Generator - 조건부 입력 (노이즈+라벨) -> `convTranspose2d` 블록 + Conditional BatchNorm
4. 모델 설계 – Discriminator - 조건부 입력(이미지 + 라벨) -> `Conv2d` 블록 + Projection Discriminator
5. 손실·옵티마이저 정의 - BCELoss 또는 WGAN‑GP → Adam/AdamW
6. 학습 루프 구현 - 에포크, 디스크리미네이터·제너레이터 순차 업데이트, 로깅, 체크포인트 저장
7. 중간·최종 결과 시각화 - 클래스별 샘플 이미지 그리드, 학습 로스·시각화
8. 정량적 평가 - FID(Fréchet Inception Distance), IS(Inception Score) 혹은 K‑NN 기반 평가
9. 모델 저장·로드 & 인퍼런스 함수 - .pth 저장·로드, generate(class_id, n_samples)

아래에서 각 단계별 핵심 코드를 한 줄씩 짚어 보겠습니다.

### 프로젝트 할 일 목록 (ToDO List)

1. 환경 설정 및 데이터 로드: 필요한 라이브러리를 임포트하고 Fashion-MNIST 데이터셋을 불러옵니다.
2. 데이터 전처리 및 시각화: 데이터를 정규화하고, 샘플 이미지를 확인하여 데이터셋의 구성을 이해합니다.
3. cGAN 모델 설계:
   * 생성자 (Generator): 노이즈 벡터와 조건부 레이블을 입력받아 이미지를 생성하는 모델을 설계합니다.
   * 판별자 (Discriminator): 이미지와 조건부 레이블을 입력받아 해당 이미지가 진짜인지 가짜인지 판별하는 모델을
         설계합니다.
4. 학습 환경 설정: 모델, 손실 함수, 옵티마이저를 초기화하고 학습에 필요한 하이퍼파라미터를 정의합니다.
5. cGAN 모델 학습: 생성자와 판별자를 번갈아 학습시키는 훈련 루프를 구현하고, 주기적으로 생성된 이미지를 저장하여 학습
      과정을 모니터링합니다.
6. 조건부 이미지 생성 및 평가: 학습된 모델을 사용하여 각 클래스별 이미지를 생성하고, 생성된 이미지의 품질을 시각적으로 평가합니다.

## 환경 설정 및 데이터 로드

- Python 3.10.19+
- PyTorch 2.5.1+ (CUDA 12 권장)
- torchvision, tqdm, matplotlib, numpy
- 정량적 평가용: torch-fidelity (FID/IS), scikit-learn (K‑NN)

In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import pandas as pd

In [27]:
# device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [28]:
# ============================================
# Dataset 및 DataLoader 설정
# ============================================
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # [-1, 1] 범위로 정규화
])

train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# 배치 형태 확인
imgs, lbls = next(iter(train_loader))
print(f'imgs.shape: {imgs.shape}, lbls.shape: {lbls.shape}')
# -> torch.Size([128, 1, 28, 28]) torch.Size([128])

# 클래스 이름 (FashionMNIST 클래스)
idx_to_class = {i: class_name for i, class_name in enumerate(train_dataset.classes)}
print(f'idx_to_class : {idx_to_class}')
print(len(idx_to_class))

imgs.shape: torch.Size([64, 1, 28, 28]), lbls.shape: torch.Size([64])
idx_to_class : {0: 'T-shirt/top', 1: 'Trouser', 2: 'Pullover', 3: 'Dress', 4: 'Coat', 5: 'Sandal', 6: 'Shirt', 7: 'Sneaker', 8: 'Bag', 9: 'Ankle boot'}
10


## cGAN 모델 정의

조건부 GAN에서 **Generator**는 두 가지 입력을 받습니다.

1. **노이즈 벡터** `z ∈ ℝ^{latent_dim}` (예: 100 차원, 표준 정규분포)
2. **클래스 라벨** `y ∈ {0,…,9}` → 임베딩 → `y_emb ∈ ℝ^{embed_dim}`

본 구현에서는 **Concatenation 방식**을 사용하여 노이즈와 레이블 임베딩을 결합합니다.

두 가지 Generator를 비교합니다:
- **GeneratorConcat**: 기본 concat 방식
- **GeneratorProjection**: Projection Discriminator와 쌍을 이루는 구조

In [50]:
# ============================================
# 생성자 (Generator) 클래스 : concat 방식 
# ============================================
class GeneratorConcat(nn.Module):
    def __init__(self, z_dim, num_classes, img_size, channels):
        super(GeneratorConcat, self).__init__()
        self.img_size = img_size
        self.channels = channels
        
        # 레이블을 위한 임베딩 레이어
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        
        # 노이즈와 임베딩된 레이블을 합친 벡터를 처리하는 모델
        self.model = nn.Sequential(
            # 입력 크기 : z_dim + num_classes
            # 초기 이미지를 만들기 위해 7x7 크기의 256개의 채널로 변환
            nn.Linear(z_dim + num_classes , 256 * 7* 7),
            nn.ReLU(),
            nn.Unflatten(1,(256,7,7)), # (배치, 256, 7, 7),
            
            # 14x14 로 업샘플링
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1, bias=False), #(배치, 128, 14, 14)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            # 28x28 업샘플링
            nn.ConvTranspose2d(128,channels, kernel_size=4, stride=2, padding=1, bias=False), #(배치, channels, 28, 28)
            
            # 출력 범위를 -1 ~ 1 정규화
            nn.Tanh()
        )
        
    def forward(self, z, labels):
        # 레이블 임베딩
        c = self.label_embedding(labels)
        # 노이즈와 레이블 결합
        x = torch.cat([z,c],1)
        # 이미지 생성
        output = self.model(x)
        return output

FashionMNIST는 28×28 크기이므로 Generator의 ConvTranspose2d 구조가 최적이라 판단하여서 Generator 클래스를 작성하였습니다.

Generator 는 1x1 또는 4x4 같이 아주 작은 feature map에서 차례대로 업샘플링하여 최종 크기를 맞춰주어야 합니다.

`ConvTranspose2d` ("deconvolution") 은 학습 가능한 필터로 **stride = 2, kernel = 4, padding =1** 처럼 한 단계씩 해당도를 두 배로 늘리면서 공간 구조를 보존할 수 있습니다.

4개의 stride-2 ConvTRranspose2d 를 쌓으면 1->2->4->8->16-> 32->28 (마지막 레이어 조정)와 같이 28x28 을 정확히 만들 수 있어, cGAN 에서 효율적인 구조가 될 것이라고 판단하였습니다.

In [51]:
# ============================================
# 판별자 (Discriminator) 클래스 : concat 방식
# ============================================
class DiscriminatorConcat (nn.Module):
    def __init__(self, num_classes, img_size, channels):
        super(DiscriminatorConcat , self).__init__()
        
        # 인스턴스 변수로 저장
        self.img_size = img_size
        self.channels = channels
        
        # 레이블을 위한 임베딩 layer
        # 임베딩된 벡터는 이미지와 같은 크기로 변환되어 채널로 합쳐진다.
        self.label_embedding = nn.Embedding(num_classes,img_size*img_size)
        
        # 이미지와 임베딩된 레이블을 합친 입력을 처리하는 모델
        self.model = nn.Sequential(
            # 입력 채널 : channels + 1 (이미지 채널 + 레이블 채널)
            nn.Conv2d(channels +1 , 64, kernel_size=4, stride=2, padding=1), #(배치, 64, 14, 14)
            nn.LeakyReLU(0.2),
            
            nn.Conv2d(64 , 128, kernel_size=4, stride=2, padding=1, bias=False), #(배치, 128, 7, 7)
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            
            # 최종 결과를 하나의 값으로 출력
            nn.Conv2d(128, 1, kernel_size=7, stride=1, padding=0), #(배치, 1, 1, 1)
            
            # BCEWithLogitsLoss 를 사용할 것이므로 Sigmoid 는 생략
        )
        
    def forward(self, x, labels):
        # 레이블 임베딩 및 이미지 크기로 변환
        c = self.label_embedding(labels)
        c = c.view(c.shape[0], self.channels, self.img_size, self.img_size)
        
        # 이미지 레이블 결합
        x = torch.cat([x,c], 1)
        
        # 판별 결과 출력
        output = self.model(x)
        return output.view(-1,1).squeeze(1) # (배치,) 크기의 텐서로 변환


FashionMNIST 는 이미지 데이터이므로 Conv 기반 Discriminator 가 최적이라 판단하여 클래스 작성하였습니다.

1. 소형 이미지에 효율적 : 28x28 픽셀에서 작은 receptive field 와 층별 다운 샘플링으로 세밀한 디테일 보존하며 학습 가능

2. 공간 특징 학습 : Conv2d 가 이미지 공간적 연관성(spatial corrlation) 을 효과적으로 파악합니다.

3. 파라미터 효율성 : 필터 공유로 FC 대비 파라미터 수가 감소하여 빠른 학습과 일반화에 용이

Conditional BatchNorm

- **조건부 배치 정규화(CBND)**: 라벨별 `γ, β` 파라미터를 학습하고, `nn.BatchNorm2d` 대신 `ConditionalBatchNorm2d` 구현.
- 이 방법은 **Projection GAN**이나 **Self‑Attention GAN**에 자주 쓰이며, 이미지 품질을 약간 향상시킵니다.

> **참고**: 여기서는 기본 Concatenation 방식만 구현하되, `ConditionalBatchNorm2d` 클래스를 별도 코드 블록에 넣어 `if use_cbn:` 형태로 스위치할 수 있게 합니다.

최신 기법  Projection Discriminator (Miyato et al., 2018)를 사용하여 조건부 정보를 더 효율적으로 하기 위해 모델을 생성하였습니다.

Projection 방식은 gradient flow를 개선하여 학습 안정성을 높입니다.

기존 concat 방식 Discriminator와의 성능 차이를 비교해보겠습니다.

In [52]:
class GeneratorProjection(nn.Module):
    def __init__(self, latent_dim=100, embed_dim=50, img_shape=(1, 28, 28)):
        super(GeneratorProjection, self).__init__()
        self.img_shape = img_shape
        self.embed_dim = embed_dim
        
        self.label_emb = nn.Embedding(10, embed_dim)
        
        self.init_size = 7
        self.fc = nn.Sequential(
            nn.Linear(latent_dim + embed_dim, 128 * 7 * 7),
            nn.BatchNorm1d(128 * 7 * 7),
            nn.LeakyReLU(0.2, inplace=True)
        )
        
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.ConvTranspose2d(64, img_shape[0], 4, 2, 1),
            nn.Tanh()
        )
    
    def forward(self, noise, labels):
        label_emb = self.label_emb(labels)
        gen_input = torch.cat((noise, label_emb), dim=1)
        out = self.fc(gen_input)
        out = out.view(out.size(0), 128, 7, 7)
        img = self.deconv(out)
        return img
    
class DiscriminatorProjection(nn.Module):
    """
    Projection Discriminator (Miyato et al., 2018)
    f(x) = h(x) + <ψ(y), φ(x)>
    where:
        - h(x): scalar output from CNN on image
        - ψ(y): label embedding (size = embed_dim)
        - φ(x): image feature vector (size = embed_dim)
    """
    def __init__(self, embed_dim=50, img_shape=(1, 28, 28)):
        super(DiscriminatorProjection, self).__init__()
        self.embed_dim = embed_dim
        self.label_emb = nn.Embedding(10, embed_dim)

        # CNN backbone (downsample 28->14->7)
        self.conv = nn.Sequential(
            nn.Conv2d(img_shape[0], 64, kernel_size=4, stride=2, padding=1),   # 1x28x28 -> 64x14x14
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),            # 64x14x14 -> 128x7x7
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Final feature -> scalar (h(x))
        self.adv_head = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 1)   # scalar real/fake logit
        )

        # Feature projection to embed_dim (φ(x))
        self.proj = nn.Linear(128 * 7 * 7, embed_dim, bias=False)

    def forward(self, img, labels):
        # img : (B,1,28,28), labels : (B,)
        feats = self.conv(img)                 # (B,128,7,7)
        feats_flat = feats.view(feats.size(0), -1)   # (B,128*7*7)

        # scalar logit from image alone
        out_adv = self.adv_head(feats_flat)    # (B,1)

        # projection term <ψ(y), φ(x)>
        phi = self.proj(feats_flat)            # (B, embed_dim)
        psi = self.label_emb(labels)           # (B, embed_dim)

        proj = torch.sum(phi * psi, dim=1, keepdim=True)   # (B,1)

        # Final discriminator logit
        out = out_adv + proj
        return out.squeeze()   # (B,)

In [53]:
# 파라미터 설정
params = {'num_classes': len(idx_to_class),
          'nz':100,
          'input_size':(1,28,28)}

In [54]:
# ============================================
# 모델 인스턴스 생성 및 구조 확인
# ============================================

# 1. Concat 방식
print("="*50)
print("1. Concatenation 방식")
print("="*50)
gen_concat = GeneratorConcat(z_dim=params['nz'], num_classes=params['num_classes'], 
                              img_size=params['input_size'][1], channels=params['input_size'][0]).to(device)
disc_concat = DiscriminatorConcat(num_classes=params['num_classes'], img_size=params['input_size'][1], 
                                   channels=params['input_size'][0]).to(device)
print("\n[Generator Concat]")
print(gen_concat)
print("\n[Discriminator Concat]")
print(disc_concat)

# 2. Projection 방식
print("\n" + "="*50)
print("2. Projection 방식")
print("="*50)
gen_proj = GeneratorProjection(latent_dim=params['nz'], embed_dim=50, 
                                img_shape=params['input_size']).to(device)
disc_proj = DiscriminatorProjection(embed_dim=50, img_shape=params['input_size']).to(device)
print("\n[Generator Projection]")
print(gen_proj)
print("\n[Discriminator Projection]")
print(disc_proj)

1. Concatenation 방식

[Generator Concat]
GeneratorConcat(
  (label_embedding): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=110, out_features=12544, bias=True)
    (1): ReLU()
    (2): Unflatten(dim=1, unflattened_size=(256, 7, 7))
    (3): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(128, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): Tanh()
  )
)

[Discriminator Concat]
DiscriminatorConcat(
  (label_embedding): Embedding(10, 784)
  (model): Sequential(
    (0): Conv2d(2, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negat

## 손실·옵티마이저 정의

> `BCELoss` 로 시작하고, 학습이 잘 안 될 경우 `WGAN‑GP` 로 전환을 권장합니다.

이 문장에서 학습 중 자동 전환을 하면 더 좋아질까 고민하였습니다.

결과는 **학습 중 자동 전환은 권장하지 않습니다.**

왜 학습 중 전환이 안 좋은가?

1. 손실 함수가 다름: BCE와 WGAN-GP는 다른 최적화 목표
    - 중간에 바꾸면 기존 학습 진행이 무의미해짐
2. 옵티마이저 상태: Adam의 momentum 등이 초기화
3. 불안정성: 전환 시점에서 학습이 더 불안정해질 수 있다

권장 방법으로는 각각 별도로 학습하고 비교하겠습니다.

### 학습 실패 판단 기준
- Mode collapse: 같은 이미지만 반복 생성
- D loss → 0, G loss 발산: 불균형
- 10-20 epoch 이상 개선 없음
- 생성 이미지 품질 저하

으로 확인하겠습니다.

In [55]:
# ============================================
# 손실 함수 및 옵티마이저 설정
# ============================================

# 학습 하이퍼파라미터
lr = 2e-4
b1, b2 = 0.5, 0.999

# 옵티마이저 (각 모델별)
optimizers = {
    'concat' : {
        'G' : torch.optim.Adam(gen_concat.parameters(), lr=lr, betas=(b1,b2)),
        'D' : torch.optim.Adam(disc_concat.parameters(), lr=lr, betas=(b1,b2))
    },
    'projection' : {
        'G': torch.optim.Adam(gen_proj.parameters(), lr=lr, betas=(b1, b2)),
        'D': torch.optim.Adam(disc_proj.parameters(), lr=lr, betas=(b1, b2))
    }
}

In [56]:
# ============================================
# BCE 손실:[ log D(x,y) ] + [ log(1 - D(G(z,y), y)) ]
# WGAN‑GP 손실: D(real) - D(fake) + λ * GP
# ============================================
# BCE 손실 함수
adversarial_loss = nn.BCEWithLogitsLoss()

# WGAN-GP 설정
lambda_gp = 10.0

def gradient_penalty(D, real, fake, labels):
    """WGAN-GP Gradient Penalty 계산"""
    alpha = torch.rand(real.size(0), 1, 1, 1, device=real.device)
    interpolates = (alpha * real + (1 - alpha) * fake).requires_grad_(True)
    d_interpolates = D(interpolates, labels)
    fake_grad = torch.ones(d_interpolates.size(), device=real.device)
    
    gradients = torch.autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake_grad,
        create_graph=True,
        retain_graph=True,
        only_inputs=True)[0]
    
    gradients = gradients.view(gradients.size(0), -1)
    gp = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gp

## 환경 설정 및 데이터 로드

> **Tip** – `log_interval` 를 `len(train_loader)//10` 정도로 잡으면 매 에포크마다 10번 정도 로스가 출력돼 학습 진행 상황을 빠르게 파악할 수 있습니다.

#### WGAN-GP 차이점

1. **Discriminator를 `n_critic` (예: 5)번 더 자주 업데이트**
   - Generator 1번 업데이트 당 Discriminator 5번 업데이트

2. **Discriminator Loss**:
   ```python
   d_loss = -real_validity.mean() + fake_validity.mean() + λ * gradient_penalty
   ```

Real 점수는 높이고(-), Fake 점수는 낮추고(+), Gradient Penalty 추가하였습니다.

Generator Loss:
   ```python
   g_loss = -fake_validity.mean()
   ```

Fake 이미지 점수를 높이는 방향

- Sigmoid 제거: 출력이 logit (무한 범위)

In [72]:
# ============================================
# 학습 루프
# ============================================

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 학습 설정
num_epochs = 100
log_interval = len(train_loader) // 10 

# 실험할 조합
model_types = ['concat', 'projection']
loss_modes = ['BCE', 'WGAN-GP']

# 고정된 노이즈와 라벨(시각화용)
fixed_noise = torch.randn(10, 100, device=device)   # 10개 클래스당 1개씩
fixed_labels = torch.arange(10, device=device)      # 0~9


def sample_and_save(generator, fixed_noise, fixed_labels, epoch, model_type, loss_mode, device):
    """
    생성 이미지 저장 함수
    
    Args:
        generator: Generator 모델
        fixed_noise: 고정된 노이즈 벡터
        fixed_labels: 고정된 라벨
        epoch: 현재 에포크
        model_type: 모델 타입 ('concat' or 'projection')
        loss_mode: 손실 함수 ('BCE' or 'WGAN-GP')
        device: 디바이스
    """
    generator.eval()
    with torch.no_grad():
        gen_imgs = generator(fixed_noise, fixed_labels).cpu()
        gen_imgs = gen_imgs * 0.5 + 0.5  # denormalize
        grid = torchvision.utils.make_grid(gen_imgs, nrow=5, normalize=False)
        out_path = f'output/{model_type}_{loss_mode}_epoch_{epoch:04d}.png'
        torchvision.utils.save_image(grid, out_path)
    generator.train()

def train_model(model_type, loss_mode, train_loader, device, num_epochs=100, save_interval=20):
    """
    cGAN 모델 학습 함수
    
    Args:
        model_type (str): 'concat' 또는 'projection'
        loss_mode (str): 'BCE' 또는 'WGAN-GP'
        train_loader: 데이터 로더
        device: 디바이스
        num_epochs (int): 학습 에포크 수
        save_interval (int): 체크포인트 저장 간격 (기본 20)
        
     Returns:
        dict: 로스 히스토리 {'step': [], 'd_loss': [], 'g_loss': []}
    """
    print(f"\n{'='*60}")
    print(f"Training {model_type.upper()} with {loss_mode}")
    print('='*60)
    
    # 모델 선택
    if model_type == 'concat':
        generator = gen_concat
        discriminator = disc_concat
        optimizer_G = optimizers['concat']['G']
        optimizer_D = optimizers['concat']['D']
    else:
        generator = gen_proj
        discriminator = disc_proj
        optimizer_G = optimizers['projection']['G']
        optimizer_D = optimizers['projection']['D']
    
    # 고정된 노이즈와 라벨 (시각화용)
    fixed_noise = torch.randn(10, 100, device=device)
    fixed_labels = torch.arange(10, device=device)
    
    # 로그 간격
    log_interval = len(train_loader) // 10
    
    # WGAN-GP 용 설정
    n_critic = 5  # Discriminator 업데이트 횟수
    
    # 로스 기록용 리스트
    loss_history = {'step': [], 'd_loss': [], 'g_loss': []}
    global_step = 0
    
    best_g_loss = float('inf')
    best_epoch = 0
    
    # ----- epoch Loop ----- 
    for epoch in range(1, num_epochs + 1):
        epoch_g_losses = []  # 에포크별 G loss 기록
        for i, (imgs, labels) in enumerate(tqdm(train_loader, desc=f'Epoch {epoch}/{num_epochs}')):
            batch_size = imgs.size(0)
            real = imgs.to(device)
            labels = labels.to(device)
            
            # -----------------
            # Train Discriminator (n_critic번 반복)
            # -----------------
            d_iterations = n_critic if loss_mode == 'WGAN-GP' else 1
            
            for _ in range(d_iterations):
                optimizer_D.zero_grad()
            
                # Real images
                real_validity = discriminator(real, labels)
                
                # Fake images
                z = torch.randn(batch_size, 100, device=device)
                fake_imgs = generator(z, labels).detach()
                fake_validity = discriminator(fake_imgs, labels)
                
                # 손실 함수 분기
                if loss_mode == 'BCE':
                    real_loss = F.binary_cross_entropy_with_logits(
                        real_validity, torch.ones_like(real_validity))
                    fake_loss = F.binary_cross_entropy_with_logits(
                        fake_validity, torch.zeros_like(fake_validity))
                    d_loss = (real_loss + fake_loss) / 2
                else:  # WGAN-GP
                    gp = gradient_penalty(discriminator, real, fake_imgs, labels)
                    d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gp
                
                d_loss.backward()
                optimizer_D.step()
            
            # -----------------
            # Train Generator (한번만 진행)
            # -----------------
            optimizer_G.zero_grad()
            
            z = torch.randn(batch_size, 100, device=device)
            gen_imgs = generator(z, labels)
            validity = discriminator(gen_imgs, labels)
            
            # 손실 함수 분기
            if loss_mode == 'BCE':
                g_loss = F.binary_cross_entropy_with_logits(
                    validity, torch.ones_like(validity))
            else:  # WGAN-GP
                g_loss = -torch.mean(validity)
            
            g_loss.backward()
            optimizer_G.step()
            
            # ----- loss 기록 ----
            global_step += 1
            loss_history['step'].append(global_step)
            loss_history['d_loss'].append(d_loss.item())
            loss_history['g_loss'].append(g_loss.item())
            epoch_g_losses.append(g_loss.item())
            
            # ----- logging -----
            if i % log_interval == 0:
                tqdm.write(f"[{model_type.upper()}/{loss_mode}] "
                           f"[E{epoch}/{num_epochs} I{i}/{len(train_loader)}] "
                           f"D loss: {d_loss.item():.4f} | G loss: {g_loss.item():.4f}")
        
        # ----- epoch-end tasks -----
        # 샘플 이미지 저장
        sample_and_save(generator, fixed_noise, fixed_labels, epoch, model_type, loss_mode, device)
        
        # # 체크포인트 저장
        # torch.save({
        #     'epoch': epoch,
        #     'model_type': model_type,
        #     'loss_mode': loss_mode,
        #     'generator_state_dict': generator.state_dict(),
        #     'discriminator_state_dict': discriminator.state_dict(),
        #     'optimizer_G_state_dict': optimizer_G.state_dict(),
        #     'optimizer_D_state_dict': optimizer_D.state_dict(),
        #     'loss_history': loss_history,
        # }, f'checkpoints/{model_type}_{loss_mode}_latest.pth')

        # 평균 G loss 계산
        avg_g_loss = np.mean(epoch_g_losses)
    
        # 최고 성능이면 별도 저장
        if avg_g_loss < best_g_loss:
            best_g_loss = avg_g_loss
            torch.save({
                'epoch': epoch,
                'model_type': model_type,
                'loss_mode': loss_mode,
                'avg_g_loss': avg_g_loss,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
                'loss_history': loss_history,
            }, f'checkpoints/{model_type}_{loss_mode}_epoch_{epoch:04d}_best.pth')
        # 주기적 체크포인트 저장 (20, 40, 60, 80, 100)
        if epoch % save_interval == 0:
            torch.save({
                'epoch': epoch,
                'model_type': model_type,
                'loss_mode': loss_mode,
                'avg_g_loss': avg_g_loss,
                'generator_state_dict': generator.state_dict(),
                'discriminator_state_dict': discriminator.state_dict(),
                'optimizer_G_state_dict': optimizer_G.state_dict(),
                'optimizer_D_state_dict': optimizer_D.state_dict(),
                'loss_history': loss_history,
            }, f'checkpoints/{model_type}_{loss_mode}_epoch_{epoch:04d}.pth')
            print(f"Checkpoint saved at epoch {epoch}")
        
    print(f"{model_type.upper()} with {loss_mode} training completed!")
    return loss_history

In [73]:
# ============================================
# 디렉토리 생성 (한 번만 실행)
# ============================================
import os
os.makedirs('checkpoints', exist_ok=True)
os.makedirs('output', exist_ok=True)
os.makedirs('plots', exist_ok=True)
print("Directories created!")

Directories created!


In [74]:
loss_hist_concat_bce = train_model('concat', 'BCE', train_loader, device, num_epochs=100)
loss_hist_concat_wgan = train_model('concat', 'WGAN-GP', train_loader, device, num_epochs=100)
loss_hist_proj_bce = train_model('projection', 'BCE', train_loader, device, num_epochs=100)
loss_hist_proj_wgan = train_model('projection', 'WGAN-GP', train_loader, device, num_epochs=100)


Training CONCAT with BCE


Epoch 1/100:   1%|          | 6/938 [00:00<00:46, 19.94it/s]

[CONCAT/BCE] [E1/100 I0/938] D loss: 0.3575 | G loss: 1.4848


Epoch 1/100:  11%|█         | 101/938 [00:02<00:16, 51.83it/s]

[CONCAT/BCE] [E1/100 I93/938] D loss: 0.2994 | G loss: 1.2621


...

[PROJECTION/WGAN-GP] [E100/100 I837/938] D loss: 346.8818 | G loss: -12633.6572


Epoch 100/100: 100%|█████████▉| 934/938 [00:56<00:00, 17.72it/s]

[PROJECTION/WGAN-GP] [E100/100 I930/938] D loss: 161.4173 | G loss: -12463.3467


Epoch 100/100: 100%|██████████| 938/938 [00:57<00:00, 16.40it/s]


Checkpoint saved at epoch 100
PROJECTION with WGAN-GP training completed!


In [ ]:
# ============================================
# 체크포인트 로드 함수
# ============================================

def load_checkpoint(checkpoint_path, generator, discriminator, optimizer_G=None, optimizer_D=None, device='cuda'):
    """
    학습된 모델 체크포인트 로드
    
    Args:
        checkpoint_path (str): 체크포인트 파일 경로
        generator: Generator 모델
        discriminator: Discriminator 모델
        optimizer_G: Generator 옵티마이저 (선택)
        optimizer_D: Discriminator 옵티마이저 (선택)
        device (str): 디바이스
    
    Returns:
        dict: 체크포인트 정보 (epoch, model_type, loss_mode, loss_history 등)
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    
    # 모델 가중치 로드
    generator.load_state_dict(checkpoint['generator_state_dict'])
    discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
    
    # 옵티마이저 로드 (필요시)
    if optimizer_G is not None and 'optimizer_G_state_dict' in checkpoint:
        optimizer_G.load_state_dict(checkpoint['optimizer_G_state_dict'])
    if optimizer_D is not None and 'optimizer_D_state_dict' in checkpoint:
        optimizer_D.load_state_dict(checkpoint['optimizer_D_state_dict'])
    
    print(f" Checkpoint loaded: {checkpoint_path}")
    print(f"   - Epoch: {checkpoint['epoch']}")
    print(f"   - Model Type: {checkpoint.get('model_type', 'N/A')}")
    print(f"   - Loss Mode: {checkpoint.get('loss_mode', 'N/A')}")
    
    return checkpoint

In [ ]:
# ============================================
# 모든 실험 실행
# ============================================
model_types = ['concat', 'projection']
loss_modes = ['BCE', 'WGAN-GP']

for model_type in model_types:
    for loss_mode in loss_modes:
        train_model(model_type, loss_mode, train_loader, device, num_epochs=100)

print("\n All experiments completed!")

### 최종 결과 시각화

In [10]:
# ============================================
# 샘플 시각화
# ============================================

def show_samples(dataset, idx_to_class, n=10):
    """
    데이터셋 샘플 시각화
    
    Args:
        dataset: 데이터셋
        idx_to_class: 클래스 인덱스->이름 매핑 딕셔너리
        n: 표시할 샘플 수
    """
    fig, axes = plt.subplots(1, n, figsize=(n*1.2, 1.5))
    for i in range(n):
        img, label = dataset[i]
        img = img.squeeze().numpy() * 0.5 + 0.5  # denormalize
        axes[i].imshow(img, cmap='gray')
        axes[i].set_title(idx_to_class[label], fontsize=8)
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()

def visualize_classes(generator, idx_to_class, device, n_per_class=8):
    """
    클래스별 생성 이미지 그리드 시각화
    
    Args:
        generator: Generator 모델
        idx_to_class: 클래스 인덱스->이름 매핑 딕셔너리
        device: 디바이스
        n_per_class: 클래스당 생성할 이미지 수
    """
    generator.eval()
    all_imgs = []
    
    with torch.no_grad():
        for class_id in range(10):
            # 동일 라벨에 대해 여러 노이즈 샘플링
            z = torch.randn(n_per_class, 100, device=device)
            labels = torch.full((n_per_class,), class_id, dtype=torch.long, device=device)
            gen_imgs = generator(z, labels).cpu()
            gen_imgs = gen_imgs * 0.5 + 0.5  # denormalize
            all_imgs.append(gen_imgs)
    
    # (10, n_per_class, 1, 28, 28) -> (10*n_per_class, 1, 28, 28)
    grid_imgs = torch.cat(all_imgs, dim=0)
    grid = torchvision.utils.make_grid(grid_imgs, nrow=n_per_class, normalize=False, pad_value=1)
    
    plt.figure(figsize=(n_per_class*1.2, 12))
    plt.title("Conditional Generation (rows: classes, cols: samples)", fontsize=14)
    plt.imshow(grid.permute(1, 2, 0).squeeze(), cmap='gray')
    plt.axis('off')
    
    # 클래스 라벨 추가
    for i, class_id in enumerate(range(10)):
        plt.text(-5, i * 28 + 14, idx_to_class[class_id], 
                fontsize=10, ha='right', va='center')
    
    plt.tight_layout()
    plt.show()
    generator.train()


In [ ]:
# ============================================
# 로스 곡선 시각화 함수
# ============================================

def plot_loss_curve(loss_history, model_type, loss_mode, save_path='plots'):
    """
    학습 로스 곡선 시각화
    
    Args:
        loss_history (dict): {'step': [], 'd_loss': [], 'g_loss': []}
        model_type (str): 'concat' 또는 'projection'
        loss_mode (str): 'BCE' 또는 'WGAN-GP'
        save_path (str): 저장 경로
    """
    import os
    os.makedirs(save_path, exist_ok=True)
    
    plt.figure(figsize=(12, 5))
    
    # 전체 로스
    plt.subplot(1, 2, 1)
    plt.plot(loss_history['step'], loss_history['d_loss'], label='Discriminator', alpha=0.7)
    plt.plot(loss_history['step'], loss_history['g_loss'], label='Generator', alpha=0.7)
    plt.xlabel('Training Step')
    plt.ylabel('Loss')
    plt.title(f'{model_type.upper()} with {loss_mode} - Training Curve')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 이동 평균 (smoothed)
    plt.subplot(1, 2, 2)
    window = 100
    if len(loss_history['step']) > window:
        d_smooth = pd.Series(loss_history['d_loss']).rolling(window=window).mean()
        g_smooth = pd.Series(loss_history['g_loss']).rolling(window=window).mean()
        plt.plot(loss_history['step'], d_smooth, label='D (smoothed)', alpha=0.7)
        plt.plot(loss_history['step'], g_smooth, label='G (smoothed)', alpha=0.7)
    else:
        plt.plot(loss_history['step'], loss_history['d_loss'], label='Discriminator', alpha=0.7)
        plt.plot(loss_history['step'], loss_history['g_loss'], label='Generator', alpha=0.7)
    plt.xlabel('Training Step')
    plt.ylabel('Loss')
    plt.title(f'Smoothed (window={window})')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    save_file = f'{save_path}/{model_type}_{loss_mode}_loss_curve.png'
    plt.savefig(save_file, dpi=150)
    print(f"Loss curve saved: {save_file}")
    plt.show()

In [ ]:
# ============================================
# 사용 예시
# ============================================

# 실제 데이터 샘플 확인
show_samples(train_dataset, idx_to_class, n=10)

In [ ]:
# ============================================
# 클래스별 이미지 그리드
# ============================================
def visualize_classes(generator, n_per_class=8):
    """
    클래스별 생성 이미지 그리드 시각화
    
    Args:
        generator: Generator 모델
        idx_to_class: 클래스 인덱스->이름 매핑 딕셔너리
        device: 디바이스
        n_per_class: 클래스당 생성할 이미지 수  
    """
    generator.eval()
    all_imgs = []
    
    # all_labels = []
    with torch.no_grad():
        for class_id in range(10):
            # 동일 라벨에 대해 여러 노이즈 샘플링
            z = torch.randn(n_per_class, 100, device=device)
            labels = torch.full((n_per_class,), class_id, dtype=torch.long, device=device)
            gen_imgs = generator(z, labels).cpu()
            gen_imgs = gen_imgs * 0.5 + 0.5  # denormalize
            all_imgs.append(gen_imgs)
            #all_labels.extend([class_names[class_id]] * n_per_class)

    # (10, n_per_class, 1, 28, 28) -> (10*n_per_class, 1, 28, 28)
    grid_imgs = torch.cat(all_imgs, dim=0)
    grid = torchvision.utils.make_grid(grid_imgs, nrow=n_per_class, normalize=True, pad_value=1)

    plt.figure(figsize=(n_per_class, 10))
    plt.title("Conditional Generation (rows: class, cols: samples)")
    plt.imshow(grid.permute(1, 2, 0).numpy())
    plt.axis('off')
    
    # 클래스 라벨 추가
    for i, class_id in enumerate(range(10)):
        plt.text(-5, i * 28 + 14, idx_to_class[class_id], 
                fontsize=10, ha='right', va='center')
    plt.tight_layout()
    plt.show()
    generator.train()

In [ ]:
# 생성된 이미지 클래스별 시각화 (학습 후)
visualize_classes(gen_concat, idx_to_class, device, n_per_class=8)
visualize_classes(gen_proj, idx_to_class, device, n_per_class=8)

### 시각적 평가

In [ ]:
# ============================================
# 시각적 평가
# ============================================

def evaluate_generation_quality(generator, idx_to_class, device, n_samples=5):
    """
    생성 품질 시각적 평가
    
    Args:
        generator: Generator 모델
        idx_to_class: 클래스 매핑 딕셔너리
        device: 디바이스
        n_samples: 클래스당 샘플 수
    """
    generator.eval()
    
    fig, axes = plt.subplots(10, n_samples, figsize=(n_samples*2, 20))
    
    with torch.no_grad():
        for class_id in range(10):
            z = torch.randn(n_samples, 100, device=device)
            labels = torch.full((n_samples,), class_id, dtype=torch.long, device=device)
            gen_imgs = generator(z, labels).cpu()
            gen_imgs = gen_imgs * 0.5 + 0.5  # denormalize
            
            for i in range(n_samples):
                axes[class_id, i].imshow(gen_imgs[i].squeeze(), cmap='gray')
                axes[class_id, i].axis('off')
                if i == 0:
                    axes[class_id, i].set_ylabel(idx_to_class[class_id], 
                                                  fontsize=12, rotation=0, 
                                                  ha='right', va='center')
    
    plt.suptitle("Generated Images by Class", fontsize=16, y=0.995)
    plt.tight_layout()
    plt.show()
    generator.train()


def compare_real_vs_fake(generator, train_dataset, idx_to_class, device, n_samples=5):
    """
    실제 vs 생성 이미지 비교
    
    Args:
        generator: Generator 모델
        train_dataset: 실제 데이터셋
        idx_to_class: 클래스 매핑
        device: 디바이스
        n_samples: 클래스당 샘플 수
    """
    generator.eval()
    
    fig, axes = plt.subplots(10, n_samples*2, figsize=(n_samples*4, 20))
    
    # 클래스별로 실제 이미지 수집
    real_images = {i: [] for i in range(10)}
    for img, label in train_dataset:
        if len(real_images[label]) < n_samples:
            real_images[label].append(img)
        if all(len(imgs) >= n_samples for imgs in real_images.values()):
            break
    
    with torch.no_grad():
        for class_id in range(10):
            # 실제 이미지
            for i in range(n_samples):
                img = real_images[class_id][i].squeeze().numpy() * 0.5 + 0.5
                axes[class_id, i].imshow(img, cmap='gray')
                axes[class_id, i].axis('off')
                if i == 0:
                    axes[class_id, i].set_ylabel(f'{idx_to_class[class_id]}\n(Real)', 
                                                  fontsize=10, rotation=0, ha='right', va='center')
            
            # 생성 이미지
            z = torch.randn(n_samples, 100, device=device)
            labels = torch.full((n_samples,), class_id, dtype=torch.long, device=device)
            gen_imgs = generator(z, labels).cpu()
            gen_imgs = gen_imgs * 0.5 + 0.5
            
            for i in range(n_samples):
                axes[class_id, n_samples + i].imshow(gen_imgs[i].squeeze(), cmap='gray')
                axes[class_id, n_samples + i].axis('off')
                if i == 0:
                    axes[class_id, n_samples].set_ylabel('(Fake)', 
                                                          fontsize=10, rotation=0, ha='right', va='center')
    
    plt.suptitle("Real vs Generated Images", fontsize=16, y=0.995)
    plt.tight_layout()
    plt.show()
    generator.train()

In [ ]:
# ============================================
# 평가 실행 예시
# ============================================
evaluate_generation_quality(gen_concat, idx_to_class, device, n_samples=5)
compare_real_vs_fake(gen_concat, train_dataset, idx_to_class, device, n_samples=5)

# Projection 모델 평가
evaluate_generation_quality(gen_proj, idx_to_class, device, n_samples=5)
compare_real_vs_fake(gen_proj, train_dataset, idx_to_class, device, n_samples=5)

### 정량적 평가

**Fashion‑MNIST** 은 흑백 28×28 라는 특성 때문에 일반적인 `Inception Score`(IS) 를 쓰기엔 적합도가 낮습니다. 대신 아래 두 지표를 활용할 수 있습니다.

| 지표 | 설명 | 구현 방법 |
|------|------|-----------|
| **FID (Fréchet Inception Distance)** | 실제 이미지와 생성 이미지의 특징 분포 차이 (InceptionV3 활용) | `torch-fidelity` (`fid.compute_fid`) |
| **K‑NN based Precision / Recall** | Feature space(K‑NN)에서 실제와 생성 이미지간의 정밀도·재현율 측정 | `scikit-learn` KNN, 또는 `torch_fidelity` 의 `kernel_inception_distance` 옵션 |
| **Classifier Accuracy (조합)** | 사전학습된 (또는 직접 학습한) Fashion‑MNIST 분류기로 생성 이미지에 라벨을 예측 → **조건부 정확도** 측정 | 직접 `CNN` 학습 후 `accuracy` 계산 |

> **주의** – `torch-fidelity` 내부에서 `InceptionV3` 를 사용하므로, 28×28 이미지를 **Resize(299,299)** 로 자동 변환합니다. 이 과정이 오래 걸릴 수 있으니 GPU 사용을 권장합니다.

In [ ]:
# ============================================
# FID 계산
# ============================================
# 1) 실제 이미지 디렉터리와 생성 이미지 디렉터리를 준비
#    (예: real/와 fake/ 폴더에 10k PNG 저장)

# 실제 이미지 저장
real_dir = 'fid/real/'
fake_dir = 'fid/fake/'

os.makedirs(real_dir, exist_ok=True)
os.makedirs(fake_dir, exist_ok=True)

# 실제 이미지 (train_set) 저장
for idx, (img, _) in enumerate(train_dataset):
    if idx >= 10000: break
    torchvision.utils.save_image(
        img * 0.5 + 0.5,  # denorm
        os.path.join(real_dir, f'{idx:05d}.png')
    )

# 생성 이미지 저장 (클래스 별 1k씩)
for class_id in range(10):
    for i in range(1000):
        z = torch.randn(1, 100, device=device)
        label = torch.tensor([class_id], device=device)
        gen = gen_concat(z, label).cpu()
        # gen = gen_proj(z, label).cpu()
        torchvision.utils.save_image(
            gen * 0.5 + 0.5,
            os.path.join(fake_dir, f'class{class_id}_{i:05d}.png')
        )

# 2) FID 계산 (torch-fidelity)
from torch_fidelity import calculate_metrics

metrics = calculate_metrics(
    input1=real_dir,
    input2=fake_dir,
    cuda=torch.cuda.is_available(),
    verbose=True,
    isc=False, # IS는 비활성화 (Inception 필요)
    fid=True,
    kid=False
)

print(f"FID: {metrics['frechet_inception_distance']:.4f}")

클래스‑조건부 정확도 (Conditional Consistency)

1. **분류기 사전 학습** – `torchvision.models.resnet18` 을 `FashionMNIST` 로 fine‑tune(1채널 → 3채널 변환 필요).
2. **생성 이미지에 라벨 예측** – `pred = classifier(gen_img)`.
3. **정확도** `accuracy = (pred.argmax(1) == label).float().mean()`.

> **Interpretation** – 10% 가 무작위 베이스라면, **50% 이상**이면 생성 이미지가 라벨 정보를 잘 반영하고 있다는 의미입니다. (예: 70%~80% 정도면 꽤 좋은 모델)

In [ ]:
# ============================================
# 클래스‑조건부 정확도 계산
# ============================================
# 예시: 간단한 CNN classifier
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, 1, 1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*7*7, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        return self.classifier(self.features(x))

# 학습 후 저장된 모델 로드
classifier = SimpleCNN().to(device)
classifier.load_state_dict(torch.load('classifier.pth'))

# 조건부 정확도 측정
def conditional_accuracy(generator, classifier, n_samples=2000):
    generator.eval()
    classifier.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for _ in range(n_samples // 100):
            batch = 100
            z = torch.randn(batch, 100, device=device)
            labels = torch.randint(0, 10, (batch,), device=device)
            gen_imgs = generator(z, labels)
            logits = classifier(gen_imgs)
            preds = logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += batch
    return correct / total

print(f"Conditional classification accuracy: {conditional_accuracy(generator, classifier)*100:.2f}%")

### 모델 저장·로드 & 인퍼런스 함수

In [ ]:
# ============================================
# 모델 저장
# ============================================
torch.save(generator.state_dict(), 'generator.pth')
torch.save(discriminator.state_dict(), 'discriminator.pth')

# 로드 함수
def load_generator(checkpoint_path='generator.pth', device='cpu'):
    model = Generator().to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()
    return model

# 인퍼런스 함수
def generate(class_id, n_samples=16, seed=None, device='cpu'):
    if seed is not None:
        torch.manual_seed(seed)
    gen = load_generator(device=device)
    z = torch.randn(n_samples, 100, device=device)
    labels = torch.full((n_samples,), class_id, dtype=torch.long, device=device)
    with torch.no_grad():
        imgs = gen(z, labels)
    # denorm
    imgs = imgs * 0.5 + 0.5
    return imgs.cpu()

# 예시 사용
samples = generate(class_id=7, n_samples=9, seed=42)  # 스니커즈
grid = torchvision.utils.make_grid(samples, nrow=3)
plt.figure(figsize=(4,4))
plt.title("Generated Sneakers")
plt.imshow(grid.permute(1,2,0))
plt.axis('off')
plt.show()